In [2]:
!pip install ucimlrepo


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
from ucimlrepo import fetch_ucirepo 
import pandas as pd

## Dataset

In [ ]:
breast_cancer_wisconsin_original = fetch_ucirepo(id=15) 
breast_cancer_wisconsin_original.metadata

{'uci_id': 15,
 'name': 'Breast Cancer Wisconsin (Original)',
 'repository_url': 'https://archive.ics.uci.edu/dataset/15/breast+cancer+wisconsin+original',
 'data_url': 'https://archive.ics.uci.edu/static/public/15/data.csv',
 'abstract': 'Original Wisconsin Breast Cancer Database',
 'area': 'Health and Medicine',
 'tasks': ['Classification'],
 'characteristics': ['Multivariate'],
 'num_instances': 699,
 'num_features': 9,
 'feature_types': ['Integer'],
 'demographics': [],
 'target_col': ['Class'],
 'index_col': ['Sample_code_number'],
 'has_missing_values': 'yes',
 'missing_values_symbol': 'NaN',
 'year_of_dataset_creation': 1990,
 'last_updated': 'Sun Mar 10 2024',
 'dataset_doi': '10.24432/C5HP4Z',
 'creators': ['WIlliam Wolberg'],
 'intro_paper': None,
 'additional_info': {'summary': "Samples arrive periodically as Dr. Wolberg reports his clinical cases. The database therefore reflects this chronological grouping of the data. This grouping information appears immediately below, ha

In [ ]:
breast_cancer_wisconsin_original.variables 

,name,role,type,demographic,description,units,missing_values
0,Sample_code_number,ID,Categorical,None,None,None,no
1,Clump_thickness,Feature,Integer,None,None,None,no
2,Uniformity_of_cell_size,Feature,Integer,None,None,None,no
3,Uniformity_of_cell_shape,Feature,Integer,None,None,None,no
4,Marginal_adhesion,Feature,Integer,None,None,None,no
5,Single_epithelial_cell_size,Feature,Integer,None,None,None,no
6,Bare_nuclei,Feature,Integer,None,None,None,yes
7,Bland_chromatin,Feature,Integer,None,None,None,no
8,Normal_nucleoli,Feature,Integer,None,None,None,no
9,Mitoses,Feature,Integer,None,None,None,no


In [19]:
# Data (as pandas dataframes) 
X = breast_cancer_wisconsin_original.data.features 
y = breast_cancer_wisconsin_original.data.targets

X.tail()

,Clump_thickness,Uniformity_of_cell_size,Uniformity_of_cell_shape,Marginal_adhesion,Single_epithelial_cell_size,Bare_nuclei,Bland_chromatin,Normal_nucleoli,Mitoses
694,3,1,1,1,3,2.0,1,1,1
695,2,1,1,1,2,1.0,1,1,1
696,5,10,10,3,7,3.0,8,10,2
697,4,8,6,4,3,4.0,10,6,1
698,4,8,8,5,4,5.0,10,4,1


In [18]:
y.tail()

,Class
694,2
695,2
696,4
697,4
698,4


In [32]:
X.describe().round(3)

,Clump_thickness,Uniformity_of_cell_size,Uniformity_of_cell_shape,Marginal_adhesion,Single_epithelial_cell_size,Bare_nuclei,Bland_chromatin,Normal_nucleoli,Mitoses
count,699.000,699.000,699.000,699.000,699.000,683.000,699.000,699.000,699.000
mean,4.418,3.134,3.207,2.807,3.216,3.545,3.438,2.867,1.589
std,2.816,3.051,2.972,2.855,2.214,3.644,2.438,3.054,1.715
min,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000
25%,2.000,1.000,1.000,1.000,2.000,1.000,2.000,1.000,1.000
50%,4.000,1.000,1.000,1.000,2.000,1.000,3.000,1.000,1.000
75%,6.000,5.000,5.000,4.000,4.000,6.000,5.000,4.000,1.000
max,10.000,10.000,10.000,10.000,10.000,10.000,10.000,10.000,10.000


### Valores nulos:

In [25]:
X.isna().sum()

Clump_thickness                 0
Uniformity_of_cell_size         0
Uniformity_of_cell_shape        0
Marginal_adhesion               0
Single_epithelial_cell_size     0
Bare_nuclei                    16
Bland_chromatin                 0
Normal_nucleoli                 0
Mitoses                         0
dtype: int64

Since we have 16 missing values on Bare_nuclei, we will handle that.

In [26]:
X[X['Bare_nuclei'].isna()]

,Clump_thickness,Uniformity_of_cell_size,Uniformity_of_cell_shape,Marginal_adhesion,Single_epithelial_cell_size,Bare_nuclei,Bland_chromatin,Normal_nucleoli,Mitoses
23,8,4,5,1,2,NaN,7,3,1
40,6,6,6,9,6,NaN,7,8,1
139,1,1,1,1,1,NaN,2,1,1
145,1,1,3,1,2,NaN,2,1,1
158,1,1,2,1,3,NaN,1,1,1
164,5,1,1,1,2,NaN,3,1,1
235,3,1,4,1,2,NaN,3,1,1
249,3,1,1,1,2,NaN,3,1,1
275,3,1,3,1,2,NaN,2,1,1
292,8,8,8,1,2,NaN,6,10,1
